## Libaries

In [1]:
# Installations
!pip install pandas

In [3]:
import pandas as pd

## Loading the data

- What files do we have
- What is the contents of the files
- Which data is useful for us

In [4]:
# list files
%ls data

data_path = "data/"

sample_submission.csv  Train.csv                  VariableDefinitions.csv
Test.csv               unlinked_masked_final.csv


In [5]:
# Read the VariableDefinitions file
var_defs = pd.read_csv(data_path + 'VariableDefinitions.csv')
var_defs

,Variable,Definition
0,CustomerId,Unique number identifying the customer on plat...
1,TransactionStartTime,Transaction start time
2,Value,Value of transaction
3,Amount,Value of Transaction with charges
4,TransactionId,Unique transaction identifier on platform
5,BatchId,Identifier for bulk transactions being done on...
6,SubscriptionId,You can have one account with multiple subscri...
7,CurrencyCode,Country currency
8,CountryCode,Numerical geographical code of country
9,ProviderId,Source provider of Item bought


### Understanding the variables and commenting on them
#### Key variables
- **Value:** Actual amount being transacted
- **Amount:** Cumulative amount with costs
(These two variables can tell us how transaction costs affect transaction behavior e.g. When transacting large sums of money, do people split them in smaller transactions, etc)
- **IssuedDate, PaidOnDate, DueData:** Track the duration of the loan
(From this we can create 2 variables 1. LoanDuration(Days) 2. OverDue (T/F))
- **AmountLoan:** Value of the loan
- **IsDefaulted:** Was the loan repaid or not
- **InvestorId:** Does the investor/institution affect how likely one can receive a loan? Are there institutions with more defaulters?
- **TransactionStatus:** (y) -> Loan accepted or rejected

#### Might be useful
- _TransactionStartTime:_ We can get patterns of transactions, what days/time are they at peak, low etc
- _ProviderId:_ Does a certain source has more transactions/customer. Are there regular people for the provider
- _ProductId:_ Is a certain product more bought. What are the trends and patterns?
- _ProductCategory_ (Same as above)
- _ChannelId:_ Is there a preferred channel of transacting?
- _InvestorId:_ Are there financial services that are more preferred for loans?

#### Sensitive variables
- CustomerId: Do we want our model to associate a customer with some fixed prediction even if their behavior might change. This could create a potential bias.

#### Not necessarily useful
- TransactionId (But can we find transactions that are anomalous?)
- BatchId
- SubscriptionID
- CurrencyCode
- CountryCode

In [6]:
# Load the training and test datasets
training_data = pd.read_csv(data_path+'Train.csv')
training_data.head(5)

,CustomerId,TransactionStartTime,Value,Amount,TransactionId,BatchId,SubscriptionId,CurrencyCode,CountryCode,ProviderId,...,LoanId,PaidOnDate,IsFinalPayBack,InvestorId,DueDate,LoanApplicationId,PayBackId,ThirdPartyId,IsThirdPartyConfirmed,IsDefaulted
0,CustomerId_27,2018-09-21 12:17:39,550.0,-550.0,TransactionId_1683,BatchId_641,SubscriptionId_2,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CustomerId_27,2018-09-25 09:20:29,550.0,-550.0,TransactionId_2235,BatchId_820,SubscriptionId_2,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CustomerId_27,2018-09-25 10:33:31,550.0,-550.0,TransactionId_1053,BatchId_210,SubscriptionId_4,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CustomerId_27,2018-09-27 10:26:41,1000.0,-1000.0,TransactionId_2633,BatchId_876,SubscriptionId_4,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CustomerId_27,2018-09-27 12:44:21,500.0,-500.0,TransactionId_71,BatchId_1362,SubscriptionId_4,UGX,256,ProviderId_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
len(training_data)

2100

In [7]:
test_data = pd.read_csv(data_path+'Test.csv')
test_data.head(5)

,CustomerId,TransactionStartTime,Value,Amount,TransactionId,BatchId,SubscriptionId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,TransactionStatus,IssuedDateLoan,LoanId,InvestorId,LoanApplicationId,ThirdPartyId
0,CustomerId_310,2019-03-31 13:33:05,14000.0,-14000.0,TransactionId_925,BatchId_1144,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_7,airtime,ChannelId_1,1,2019-03-31 13:33:04,LoanId_1027,InvestorId_1,LoanApplicationId_825,ThirdPartyId_1175
1,CustomerId_243,2019-03-31 15:04:09,1000.0,-1000.0,TransactionId_1080,BatchId_1214,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_8,data_bundles,ChannelId_1,1,2019-03-31 15:04:08,LoanId_768,InvestorId_1,LoanApplicationId_68,ThirdPartyId_604
2,CustomerId_142,2019-03-31 17:31:11,2500.0,-2500.0,TransactionId_2315,BatchId_2150,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_7,airtime,ChannelId_1,1,2019-03-31 17:31:09,LoanId_1067,InvestorId_1,LoanApplicationId_1223,ThirdPartyId_1521
3,CustomerId_142,2019-03-31 17:32:15,500.0,-500.0,TransactionId_1466,BatchId_1071,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_7,airtime,ChannelId_1,1,2019-03-31 17:32:14,LoanId_202,InvestorId_1,LoanApplicationId_633,ThirdPartyId_406
4,CustomerId_142,2019-03-31 17:34:41,1000.0,-1000.0,TransactionId_337,BatchId_2477,SubscriptionId_7,UGX,256,ProviderId_1,ProductId_7,airtime,ChannelId_1,1,2019-03-31 17:34:40,LoanId_533,InvestorId_1,LoanApplicationId_309,ThirdPartyId_302


## Exploring the data

### Cursory Examination

In [8]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CustomerId             2100 non-null   object 
 1   TransactionStartTime   2100 non-null   object 
 2   Value                  2100 non-null   float64
 3   Amount                 2100 non-null   float64
 4   TransactionId          2100 non-null   object 
 5   BatchId                2100 non-null   object 
 6   SubscriptionId         2100 non-null   object 
 7   CurrencyCode           2100 non-null   object 
 8   CountryCode            2100 non-null   int64  
 9   ProviderId             2100 non-null   object 
 10  ProductId              2100 non-null   object 
 11  ProductCategory        2100 non-null   object 
 12  ChannelId              2100 non-null   object 
 13  TransactionStatus      2100 non-null   int64  
 14  IssuedDateLoan         1488 non-null   object 
 15  Amou

In [9]:
training_data.describe()

,Value,Amount,CountryCode,TransactionStatus,AmountLoan,IsFinalPayBack,IsThirdPartyConfirmed,IsDefaulted
count,2.100000e+03,2.100000e+03,2100.0,2100.000000,1.488000e+03,1488.000000,1488.000000,1488.000000
mean,5.302947e+04,-5.291354e+04,256.0,0.708571,1.182725e+04,0.731855,0.982527,0.119624
std,1.539498e+05,1.539653e+05,0.0,0.454529,7.666506e+04,0.443143,0.131070,0.324630
min,1.000000e+02,-2.500000e+06,256.0,0.000000,5.000000e+01,0.000000,0.000000,0.000000
25%,1.000000e+03,-1.200000e+04,256.0,0.000000,1.113750e+03,0.000000,1.000000,0.000000
50%,3.500000e+03,-3.500000e+03,256.0,1.000000,2.800000e+03,1.000000,1.000000,0.000000
75%,1.200000e+04,-1.000000e+03,256.0,1.000000,1.050000e+04,1.000000,1.000000,0.000000
max,2.500000e+06,-1.000000e+02,256.0,1.000000,2.760000e+06,1.000000,1.000000,1.000000


In [19]:
training_data.isnull().sum()

CustomerId                 0
TransactionStartTime       0
Value                      0
Amount                     0
TransactionId              0
BatchId                    0
SubscriptionId             0
CurrencyCode               0
CountryCode                0
ProviderId                 0
ProductId                  0
ProductCategory            0
ChannelId                  0
TransactionStatus          0
IssuedDateLoan           612
AmountLoan               612
Currency                 612
LoanId                   612
PaidOnDate               612
IsFinalPayBack           612
InvestorId               612
DueDate                  614
LoanApplicationId        617
PayBackId                612
ThirdPartyId             614
IsThirdPartyConfirmed    612
IsDefaulted              612
dtype: int64

In [24]:
# Number of datapoints
len(training_data) * len(training_data.columns)

56700

In [25]:
# Number of null values
training_data.isnull().values.sum()

7965

In [28]:
# Get columns with numeric data
numeric_training_data = training_data.select_dtypes('number')
numeric_training_data

,Value,Amount,CountryCode,TransactionStatus,AmountLoan,IsFinalPayBack,IsThirdPartyConfirmed,IsDefaulted
0,550.0,-550.0,256,0,NaN,NaN,NaN,NaN
1,550.0,-550.0,256,0,NaN,NaN,NaN,NaN
2,550.0,-550.0,256,0,NaN,NaN,NaN,NaN
3,1000.0,-1000.0,256,0,NaN,NaN,NaN,NaN
4,500.0,-500.0,256,0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2095,2000.0,-2000.0,256,1,2370.0,1.0,1.0,0.0
2096,5000.0,-5000.0,256,1,5925.0,1.0,1.0,0.0
2097,5000.0,-5000.0,256,1,7100.0,1.0,1.0,0.0
2098,1500.0,-1500.0,256,1,1733.0,1.0,1.0,0.0


In [30]:
# Numeric columns with null
numeric_training_data.isnull().values.sum()

2448

In [35]:
numeric_training_data.isnull().sum()

Value                      0
Amount                     0
CountryCode                0
TransactionStatus          0
AmountLoan               612
IsFinalPayBack           612
IsThirdPartyConfirmed    612
IsDefaulted              612
dtype: int64

### Distribution of data

2448